In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy

# numpy基础
数据处理库。核心是多维数组NDarray
底层使用C实现，效率高。

## NDArray
numpy中最重要的类。多维数据包装器

多维数组的创建、修改以及多维数组的属性

### 创建
从数组创建或使用内置函数

In [ ]:
arr1 = np.array([1, 2, 3, 4, 5])
arr2 = np.zeros([2, 2, 3], dtype = np.float32)
arr3 = np.ones([2, 2])
print(arr1, type(arr1))
print(arr2)
print(arr3)

### 修改
主要包括维度变换和拼接

In [ ]:
arr1 = np.zeros([1, 1, 4],  dtype=np.int32)
arr2 = np.ones([5, 1, 4], dtype=np.int32)
arr = np.concatenate([arr1, arr2], axis=0)
print('concatenate:', arr)

print('expand1:', np.expand_dims(arr1, 3))
print('expand2', arr2[ :, :, np.newaxis, :])

### 获取数组属性

In [ ]:
arr = np.array([[1, 1, 4, 5, 1, 4], [1, 9, 1, 9, 8, 10]])
print('size:', arr.size)
print('shape:', arr.shape)

### 切片和筛选
切片和python list一致



In [ ]:
arr = np.array([[1, 1, 4, 5, 1], 
                [4, 1, 9, 1, 9],
                [8, 1, 0, 1, 1], 
                [4, 5, 1, 4, 1], 
                [9, 1, 9, 8, 10]])

print('single choose')
print(arr[1])
print(arr[1, 0])
print(arr[[1, 0], [2, 3]])

print('slice')
print(arr[:2, :3])

print('filter')
print(arr>5)
print(arr[arr>7])



### np.where
按条件选择、替换数据

In [ ]:
condition = arr>5
print(condition)
print(np.where(condition, -1, arr))
print(np.where(condition, -1, 2))
rra = -arr
print(np.where(condition, arr, rra))

## 数据保存和加载
```python
numpy.save(file, arr, allow_pickle=True, fix_imports=True)
```
- file：要保存的文件，扩展名为 .npy，如果文件路径末尾没有扩展名 .npy，该扩展名会被自动加上。
- arr: 要保存的数组

```python
numpy.savez(file, *args, **kwds)
```
- file：要保存的文件，扩展名为 .npz，如果文件路径末尾没有扩展名 .npz，该扩展名会被自动加上。
- args: 要保存的数组，可以使用关键字参数为数组起一个名字，非关键字参数传递的数组会自动起名为 arr_0, arr_1, …　。
- kwds: 要保存的数组使用关键字名称。

In [ ]:
# 保存
a = np.array([1, 1, 4, 5, 1, 4])
b = np.array([[1, 9, 1], [9, 8, 10]])

np.save("result/a", a)
np.savez("result/ab", a=a, b=b)

In [ ]:
# 加载
a = np.load('result/a.npy')
ab = np.load('result/ab.npz')

print(a)
print(ab)
print(ab['a'])
print(ab['b'])

## Broadcast
一种自动复制填充机制，使得原本形状不同的两个array能够进行原本只有两相同形状array才能进行的操作。

广播的规则如下：

1. 从后向前，如果两数组对应维度上轴的长度相同或其中一个的轴长度为1，广播兼容，可在轴长度为1的轴上进行广播机制处理。
2. 如果两个数组的维度不同导致某个数组的前方没有维度，那么给低维度的数组前扩展提升一维，扩展维的轴长度为1,然后在扩展出的维上进行广播机制处理。

In [ ]:
'''
a: (3, 5)
b: (5)

1. extend b into (1, 5)
2. copy b in dim 0 to (3, 5)
'''
a = np.arange(1, 16).reshape([3, 5])
print(a)
b = np.array([2, 3, 4, 5, 6])
print(b)
print(a+b)

In [ ]:
"""
2: ()
a: (3, 5)

1. extend 2 into (1)
2. copy 2 in dim 0 to (5)
3. extend 2 into (1, 5)
4. copy 2 in dim 0 to (3, 5)
"""

a = np.arange(1, 16).reshape([3, 5])
print(2 * a)

In [ ]:
'''
a: (1, 5)
b: (3, 4, 1)

1. copy b in dim 2 to (3, 4, 5)
2. copy a in dim 0 to (4, 5)
3. extend a into (1, 4, 5)
4. copy a in dim 0 to (3, 4, 5)
'''

a = np.arange(1, 5)
b = np.arange(0, 12).reshape((3, 4, 1))
print(a + b)

## 线性代数

In [ ]:
# 矩阵和向量积
a = np.random.randint(5, size=(3, 3))
b = np.random.randint(5, size = (3, 3))

print(f'a:{a}\nb:{b}')

# dot根据不同情况选择，1维数组为内积，2维数组为矩阵乘，其他情况请阅读文档
print(f'dot:\n {np.dot(a, b)}')
# vdot计算逐元素乘积并求和
print(f"vdot:\n {np.vdot(a, b)}")
# inner 计算最后一维内积
print(f"inner:\n {np.inner(a, b)}")
# 展开后算ab^T
print(f"outer:\n {np.outer(a, b)}")
# matmul计算矩阵乘
print(f"outer:\n {a@b}")

### Einstein Notation
$a_i b_i = \sum_{i} a_i b_i$

$a_{ij}b_{jk} = \sum_{j}a_{ij}b_{jk}$


In [ ]:
import torch
a = np.arange(0, 9).reshape(3, 3)
print(a)
print(np.einsum("ij->ji", a))


A = torch.tensor(np.random.randint(5, size=(2, 2, 2, 5))).reshape(2, 2, 1, 2, 5)
B = torch.tensor(np.random.randint(5, size=(2, 2, 2, 5))).reshape(2, 1, 2, 2, 5)
"""
想将两者的第2个维度分别reshape为2x1和1x2的两个向量，然后计算外积，得到一个2x2的kernel
"""
print(A.shape, B.shape)
res = torch.einsum(
    "ijk...,iko...->ijo...", [A, B]
)
print(res.shape)

## numpy使用原则
尽量向量化所有操作，让numpy可以自动并行

1. 减少遍历操作
2. 尽量使用内置方法和函数
3. 善用广播

In [ ]:
import time

s_time = time.time()
a = np.array([i for i in range(1919810)])
print(time.time()-s_time)

s_time = time.time()
a = np.linspace(0, 1919810, 1919811)
print(time.time() - s_time)

In [ ]:
a = np.random.random(1919810)
b = np.random.random(1919810)

s_time = time.time()
print(a@b)
print(time.time() - s_time)

s_time = time.time()
res = 0
for i in range(len(a)):
    res += a[i]*b[i]
print(res)
print(time.time() - s_time)

In [ ]:
samples = np.random.random((1145, 14))



s_time = time.time()

dists = []
for p1 in samples:
    for p2 in samples:
        dists.append(np.linalg.norm(p1-p2))
print(np.mean(dists))

print(time.time() - s_time)

s_time = time.time()

diff = samples[:, np.newaxis, :] - samples[np.newaxis, :, :]
dist = np.linalg.norm(diff, axis = -1)
print(np.mean(dist))

print(time.time() - s_time)

# Matplotlib绘图
以曲线为例
## 基础

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

X = np.linspace(-np.pi, np.pi, 256, endpoint=True)
C, S = np.cos(X), np.sin(X)

plt.plot(X, C)
plt.plot(X, S)

plt.show()

In [ ]:
plt.figure(figsize=(8, 6), dpi=80)


X = np.linspace(-np.pi, np.pi, 30, endpoint=True)
C, S, T, A = np.cos(X), np.sin(X), np.arctan(X), np.arccos(X)

plt.subplot(2, 2, 1)
# 绘制余弦曲线，使用蓝色的、连续的、宽度为 1 （像素）的线条
plt.plot(X, C, color="blue", linewidth=1.0, linestyle="-")

plt.subplot(2, 2, 2)
# 绘制正弦曲线，使用绿色的、连续的、宽度为 1 （像素）的线条
plt.plot(X, S, color="green", linewidth=4.0, linestyle="-.")

plt.subplot(2, 2, 3)
plt.scatter(X, T, color='red', alpha=0.5)

plt.subplot(2, 2, 4)
plt.bar(X, T, color="yellow", alpha=0.5)

# 在屏幕上显示
plt.show()

In [ ]:
X = np.linspace(-np.pi, np.pi, 256, endpoint=True)
C, S = np.cos(X), np.sin(X)

plt.plot(X, C, label = 'sin')
plt.plot(X, S, label='cos')

# 修改标签
plt.xticks(
    [-np.pi, -np.pi / 2, 0, np.pi / 2, np.pi],
    [r"$-\pi$", r"$-\pi/2$", r"$0$", r"$+\pi/2$", r"$+\pi$"],
)

plt.yticks([-1, 0, +1], [r"$-1$", r"$0$", r"$+1$"])

# 修改坐标轴位置
ax = plt.gca()
ax.spines["right"].set_color("none")
ax.spines["top"].set_color("none")
ax.xaxis.set_ticks_position("bottom")
ax.spines["bottom"].set_position(("data", 0))
ax.yaxis.set_ticks_position("left")
ax.spines["left"].set_position(("data", 0))

# 显示图例
plt.legend(loc="upper left")

# 标注
t = 2 * np.pi / 3
plt.scatter(
    [
        t,
    ],
    [
        np.sin(t),
    ],
    s=100,
    color="red",
)
plt.annotate(
    r"$\sin(\frac{2\pi}{3})=\frac{\sqrt{3}}{2}$",
    xy=(t, np.sin(t)),
    xycoords="data",
    xytext=(+10, +30),
    textcoords="offset points",
    arrowprops= dict(arrowstyle = "->"),
)


plt.show()

# Scipy
> an open-source software for mathematics, science, and engineering.
## Document
https://docs.scipy.org/doc/scipy/reference/index.html
## constants

In [ ]:
import scipy
import scipy.constants
print(dir(scipy.constants))

print(scipy.constants.Avogadro)

## optimizer

In [ ]:
from math import cos

def eqn(x):
    return x + cos(x)

myroot = scipy.optimize.root_scalar(eqn, method='newton', x0=0.1)

print(myroot)

In [ ]:
def eqn(x):
    return (x[0]-3)**2 + (x[1]-1)**2

scipy.optimize.minimize(eqn, x0=[0, 0], method='CG')

## Graph

In [ ]:
from scipy.sparse.csgraph import connected_components, dijkstra, floyd_warshall
from scipy.sparse import csr_matrix

arr = np.array([[0, 1, 1], 
                [1, 0, 1111], 
                [2, 1111, 0]])

newarr = csr_matrix(arr)

print(connected_components(newarr))
print(dijkstra(newarr, return_predecessors=True, indices=2))
print(floyd_warshall(newarr, return_predecessors=True))

## statistics
### 随机变量相关

连续随机变量对象有下列方法：
- rvs：采样
- pdf：密度函数
- cdf：累积分布函数
- sf：残存函数
- ppf：累积分布的逆
- isf：残存函数的逆
- fit：最大似然估计求参数

离散分布的简单方法大多数与连续分布很类似，但是pdf被更换为密度函数pmf。

In [ ]:
# 从常用分布中采样
from scipy import stats
rv = stats.norm(
    loc=0,
    scale=1
)
print(rv.rvs(size=10))
print(rv.pdf(1))

data = stats.norm.rvs(loc=114, scale=514, size=1000000)
print(stats.norm.fit(data))

### 假设检验
常用的如下：
- 单样本均值检验
- 两独立样本均值检验
    - 方差相等
    - 方差不相等
    - 检验方差
- 配对样本均值检

In [ ]:
from scipy.stats import ttest_ind, ttest_1samp, levene, ttest_rel
import random

v1 = np.random.normal(size=1000)
v2 = np.random.normal(loc=1, size=1000)

# 单总体方差
res = ttest_1samp(v1, 0)
print(res)

# 两独立总体
res = ttest_ind(v1, v2, equal_var=True)
print(res)

# 两总体方差
res = levene(v1, v2)
print(res)

v_pair = [i + random.uniform(-1, 1) for i in v1]
# 配对样本均值
res= ttest_rel(v1, v_pair)
print(res)

# Pytorch
- 拥有类似于NumPy的张量计算系统，能在GPU等硬件上加速
- 自动微分系统的计算图在运行中构建（即动态图系统，区别于tensorflow先定义再建图的静态图系统）

动态图带来的好处在具有复杂控制条件的网络中尤其明显。例如，pytorch可以直接进行如下实现：
```python
def forward(self, x)
    for i in range(5):
        cond = random.randint(1, 5)
        if cond == 1:
            x = self.fc1(x)
        elif cond == 2:
            x = self.fc2(2)
        #.....
    return x
```
而使用tensorflow则需要使用tf.cond()进行多层嵌套


## Pytorch模型训练八股文
```python
class MyDataset(Dataset):
    def __init__(self, *args):
    
    def __getitem__(self, index):

    def __len__(self)

dataset = MyDataset(*args)
dataloader = DataLoader(dataset, *args)

class Model(nn.Modules):
    def __init__(self, *args):
        super().__init__()

    def forward(self, x):
        return output

model = Model(*args)

lossfn = Loss()
optimizer = Optimizer(model.parameters(), lr=LEARNING_RATE)

for epoch_num in range(EPOCH):
    for (data, label) in dataloader:
        output = model(data)
        loss = lossfn(output, label)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

torch.save(model, MODEL_SAVE_PATH)

```

## Pytorch的模型定义
pytorch中所有神经网络模型都由 torch.nn.Module 派生得到。在实现自己的模型时，绝大多数情况下只需要重写__init__, forward两个方法。

In [ ]:
import torch

class TestModel(torch.nn.Module):
    def __init__(self):
        super(TestModel, self).__init__()
        # 简写：super().__init__()
        self.layer = torch.nn.Linear(5, 2)

    def forward(self, data):
        return(self.layer(data))




## Pytoch梯度计算与模型优化

In [ ]:
test_model = TestModel()
data = torch.rand((1, 5))
y = test_model(data)

optimizer = torch.optim.SGD(test_model.parameters(), lr=1)
loss_fn = torch.nn.MSELoss()

print("-------------------\nbefore loss backward:\n")
print(test_model.layer._parameters["weight"])
print(test_model.layer._parameters["weight"].grad)

loss = loss_fn(torch.randn((1, 2)), y)
loss.backward()

print("-------------------\nafter loss backward:\n")
print(test_model.layer._parameters["weight"])
print(test_model.layer._parameters["weight"].grad)
print(test_model.layer._parameters["weight"].grad.shape)

print(loss.grad_fn)
print(loss.grad_fn.next_functions)

optimizer.step()

print("-------------------\nafter optimize:")

print(test_model.layer._parameters["weight"])
print(test_model.layer._parameters["weight"].grad)
print(test_model.layer._parameters["weight"].grad.shape)

optimizer.zero_grad()
print("-------------------\nafter zerograd:")

print(test_model.layer._parameters["weight"])
print(test_model.layer._parameters["weight"].grad)


with torch.no_grad():
    loss = loss_fn(torch.randn((1, 2)), y)
    print(loss.grad_fn)

## Pytorch数据集

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from tokenizers import Tokenizer
import json
import threading
from tqdm import tqdm
import math

tokenizer = Tokenizer.from_file("tokenizer/tokenizer.json")
tokenizer.enable_padding(length=256)

def deal_segment(inputlist, savelist):
    for item in inputlist:
        tmp_dict = json.loads(item)
        savelist.append(
            (
                torch.tensor(tokenizer.encode(tmp_dict["content"]).ids[:256]),
                torch.tensor(
                    [1 - tmp_dict["label"], tmp_dict["label"]], dtype=torch.float32
                ),
            )
        )

class MyDataSet(Dataset):
    def __init__(self, file: str):
        self.data = []
        with open(file, "r", encoding='utf-8') as fin:
            inputlist = list(fin)
            tlist = [threading.Thread(target=deal_segment, args = (inputlist[1000*i:1000*(i+1)], self.data,)) for i in range(math.ceil(len(inputlist)/1000))]
            for t in tqdm(tlist):
                t.start()
            for t in tlist:
                t.join()
        fin.close()
                
    def __getitem__(self, index):
        return self.data[index]
    
    def __len__(self):
        return len(self.data)

train_set = MyDataSet(file="dataset/train.jsonl")
test_set = MyDataSet(file="dataset/test.jsonl")

In [ ]:
train_loader = DataLoader(dataset=train_set, batch_size=32, shuffle=True)
test_loader = DataLoader(dataset=test_set, batch_size=32, shuffle=True)

for data, label in train_loader:
    print(data)
    print(label)
    break

## 训练实例

In [ ]:
import torch
import torch.nn as nn

# 模型定义
class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.emb = nn.Embedding(num_embeddings=50000, embedding_dim=64)
        self.layer1 = nn.Linear(256*64, 64*128)
        self.ac1 = nn.ReLU()
        self.layer2 = nn.Linear(64*128, 16*16)
        self.ac2 = nn.ReLU()
        self.out = nn.Linear(16*16, 2)
        
    def forward(self, data):
        hidden = self.emb(data).view(-1, 64*256)
        return self.out(self.ac2(self.layer2(self.ac1(self.layer1(hidden)))))
    
    
model = MyModel()

In [ ]:
from torch.optim import SGD
import torch.nn as nn

# 损失函数和优化器定义
loss_fn = nn.CrossEntropyLoss()
optimizer = SGD(model.parameters(), lr=1e-3)

In [ ]:
import wandb
import numpy as np

# 训练
wandb.finish()
wandb.init(
    # set the wandb project where this run will be logged
    project="summer_guide",
    # track hyperparameters and run metadata
    config={
        "learning_rate": 1e-3,
        "architecture": "MLP",
        "dataset": "amazon-plarity",
        "epochs": 3,
    },
)
# model.cuda()
for i in range(wandb.config['epochs']):
    for batch, (X, y) in enumerate(train_loader):
        pred = model(X.cuda())
        loss = loss_fn(pred, y.cuda())

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 20 == 0:
            wandb.log(
                {
                    "loss": loss,
                    "acc": np.mean((torch.argmax(pred.cpu(), 1) == torch.argmax(y.cpu(), 1)).numpy()),
                }
            )
    torch.save(model.state_dict(), "result/model.pt")

In [ ]:
torch.save(model.state_dict(), "result/model.pt")

In [ ]:
# 推理
# model = MyModel()
# model.load_state_dict(torch.load('result/model.pt'))
import torch
from tokenizers import Tokenizer


model = torch.load(
    "result/model.pt", map_location=torch.device("cpu"), weights_only=False
)
tokenizer = Tokenizer.from_file("tokenizer/tokenizer.json")
tokenizer.enable_padding(length=256)
# sentence = "I bought this charger in Jul 2003 and it worked OK for a while. The design is nice and convenient. However, after about a year, the batteries would not hold a charge. Might as well just get alkaline disposables, or look elsewhere for a charger that comes with batteries that have better staying power."
sentence = "My lovely Pat has one of the GREAT voices of her generation. I have listened to this CD for YEARS and I still LOVE IT. When I'm in a good mood it makes me feel better. A bad mood just evaporates like sugar in the rain. This CD just oozes LIFE. Vocals are jusat STUUNNING and lyrics just kill. One of life's hidden gems. This is a desert isle CD in my book. Why she never made it big is just beyond me. Everytime I play this, no matter black, white, young, old, male, female EVERYBODY says one thing \"Who was that singing ?\""
ids = torch.tensor(tokenizer.encode(sentence).ids)
print("origin sentence:", tokenizer.decode(ids.tolist()), '\n')
model.eval()
with torch.no_grad():
    out = torch.nn.functional.softmax(model(ids), dim=-1)
    print("probability:", out, '\n')
    print("class:", torch.argmax(out).item(), '\n')

# 作业
编写脚本，实现RNN文本情感分类任务，要求如下：
- 使用GRU或LSTM块作为RNN块（推荐手写，但也可以使用pytorch内置的类，如果想使用请阅读pytorch文档）
- RNN能够实现不定长输入，即根据输入长度判断RNN的循环次数
- 分类正确率超过80%
- 能够使用plt或wandb绘制训练过程的loss、train accuracy和test accuracy曲线
- 能够保存训练后的神经网络参数，以及加载并使用该参数进行推理